<a href="https://colab.research.google.com/github/priyal6/DL/blob/main/self_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [3]:
class SelfAttention(nn.Module):
   def __init__(self, embed_size, heads):
    super(SelfAttention , self).__init__()
    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = embed_size // heads

    assert(
        self.head_dim * heads == embed_size
    ), "Embedding size needs to be divisible by heads"

    self.query = nn.Linear(embed_size, embed_size)
    self.key = nn.Linear(embed_size, embed_size)
    self.value = nn.Linear(embed_size, embed_size)

    self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self,x):
      N, seq_length, embed_size = x.shape

      Q = self.query(x)
      K = self.key(x)
      V = self.value(x)

      Q = Q.view(N, seq_length, self.heads, self.head_dim).transpose(1, 2)
      K = K.view(N, seq_length, self.heads, self.head_dim).transpose(1, 2)
      V = V.view(N, seq_length, self.heads, self.head_dim).transpose(1, 2)

      energy = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
      attention = torch.softmax(energy, dim=-1)
      out = torch.matmul(attention, V)

      out = out.transpose(1, 2).contiguous().view(N, seq_length, embed_dim)
      out = self.fc_out(out)
      return out